In [0]:
df=spark.read.csv("/Volumes/testing/default/employee/emp.csv",              # employees.csv
                  header=True,
                  inferSchema=True)

In [0]:
df.printSchema()

In [0]:
df=df.withColumnsRenamed({'First Name':'First_Name',
                       'Gender':'Gender',
                       'Start Date':'Start_Date',
                       'Last Login Time':'Last_Login_Time',
                       'Bonus %':'Bonus_Percent',
                       'Senior Management':'Senior_Management',
                       })

In [0]:
df.write.saveAsTable("testing.default.m_bronze")

In [0]:
df.show(5)

In [0]:
df=df.dropna(subset=["First_Name"])

In [0]:
from pyspark.sql.functions import col, when

cols_to_check = ['Gender', 'Senior_Management', 'Team']
null_count = sum([when(col(c).isNull(), 1).otherwise(0) for c in cols_to_check])

df_cleaned = df.filter(null_count < 2)
df_cleaned.count()

In [0]:
df_cleaned=df_cleaned.na.fill({'Team': 'Unknown', 'Gender':'Unknown'})

In [0]:
P_df=df_cleaned.toPandas()
print("No.of Nulls :\n",P_df.isnull().sum())
print("No.of Unique values :\n",P_df.nunique())


In [0]:
df_cleaned.write.saveAsTable("testing.default.m_silver")

In [0]:
df_cleaned.printSchema()

In [0]:
from pyspark.sql.functions import avg, max, min, sum, round

team=df_cleaned.groupBy("Team").agg(
    round(avg("Salary"),2).alias("avg_salary"),
    max("Salary").alias("max_salary"),
    min("Salary").alias("min_salary"),
    sum("Salary").alias("sum_salary")
).orderBy("avg_salary",ascending=False)


In [0]:
team.write.saveAsTable("testing.default.m_gold")